# Clustering on Bernoulli Embeddings with Dynamic Context

*Yufan Zhuang*<br>
*Xuewei Du*<br>
*Columbia University*

## Introduction

Word embeddings are powerful in terms of learning latent semantic structure in language.  Recently, Rudolph and Blei (Rudolph and Blei, 2018) explained dynamic embeddings for language evolution, in which he used exponential family embeddings (Rudolph et al., 2018) which can change over time to capture the change in meanings of words over time. Here in this report, we would like to elaborate some findings on modification of the previous models, where we will have context vectors that change over time, and we will have Gaussian Mixture Model(GMM) to cluster words. We study how this new model performs on United Nations general debates data. 

### Main Contributions

Our contribution is threefold.

1. We made the contextual vectors time-dependent, such that it captures the dynamic language evolution in time.
2. We built our model in a hybrid setting that stabilizes the training process which could be generalized to other embedding training models.
3. We constructed a topic model that outperforms LDA in undirected coherence measure.

### Background

#### Word embeddings

Word embeddings can be used to represent text data. For instance, in frequency based embedding, if we do one-hot encoding on a particular text data, then we will have a vector of the same length as the number of unique words to represent each word. To represent a particular word, we can set the corresponding position in the vector to 1, and others to 0. However, more generally, we do not necessarily have to use such high dimensional vectors to represent words. We can have one vector of much lower dimension(i.e. a much shorter vector) to represent a word, and each position in that vector can be decimal numbers instead of (0,1). In probabilistic programming setting, each position in this embedding vector can be viewed as a parameter. 

#### Context vector

Each word in the observed text is not only dependent upon word embeddings but also its context. This means each word is related to a few words before and after itself. As Rudolph and Blei (Rudolph and Blei, 2018) pointed out in the paper, the typical context has a size between 2 and 10, and each word is modeled conditionally on the words before and after. 

#### Prior Work

Rudolph and Blei (Rudolph and Blei, 2018) explained how dynamic embeddings work. In particular, for each time slice, there are some observed words. Each observed word is dependent upon both its embeddings under the same time slice and its context vector which is shared among all time slices. Over time, the embeddings change while the context stays the same. 

## Model

We are proposing clustering using GMM Bernoulli embeddings and dynamic context vectors. There are practically three components to the model, Bernoulli, dynamic and GMM. 

<img src="dynamic_graph.png" style="width: 600px;"/>


Bernoulli means the Bernoulli embedding model (Rudolph and Blei, 2018). Under the Bernoulli embedding model, we have a one-hot encoded embedding vector of length V to represent each word. The corresponding position is 1 for a particular word, and 0 for other positions.  Each position in the embedding vector of each observed word is modeled using Bernoulli distribution with probability $p_{iv}$ for each $v$ in the entire vocabulary $V$. 

$$P(x_{iv}|x_{c_i}) = Bern(p_{iv})$$

The log odds $\eta_{iv} = log \frac{p_{iv}}{1-p_{iv}}$, which is a function of $p_{iv}$, can be specified as 

$$\eta_{iv} = \rho_v^T(\sum_{j \in c_i}\sum_{v'}\alpha_{v'}x_{jv'})$$ Here $\rho_v \in \mathbb{R}^K$ represents embeddings and $\alpha_{v'} \in \mathbb{R}^K$ represents context. The value K is set in advance. Therefore the log-odds is an inner-product between the embeddings and context vectors. 

Dynamic means dynamic context vector. In particular, the log odds can be specified as the inner-product between the embeddings and **time-specific** context vector. $$\eta_{iv} = \rho_v^T(\sum_{j \in c_i}\sum_{v'}\alpha_{v'}^{t_i}x_{jv'})$$ where $t_i$ is the time slice for the word $x_{iv}$. As we can see in the above graph illustration, we have different alphas for different time slices. <br>
The prior on dynamic context vectors is Gaussian random walk, which can be specified as follow: 

$$\alpha_v^0 \sim N(0, \lambda_0^{-1}I)$$ $$\alpha_v^t \sim N(\alpha_v^{t-1}, \lambda^{-1}I)$$

Furthermore, we build a GMM on the embedding vectors $\rho_v$. 

$$P(\rho_v | \pi, \mu, \Lambda) = \Pi_{k=1}^{K}(\pi_k N(\rho_v | \mu_k, \Lambda_k^{-1}))$$ 

The goal is to learn parameters $\pi$, $\mu$ and $\Lambda$. Their prior can be set as 

$$P(\pi) \sim Dir(\alpha)$$ $$P(\mu_k) \sim N(0, I)$$ $$P(\Lambda_k) \sim Wishart(a, B)$$.

In [8]:
import numpy as np
import os
import pickle
import tensorflow as tf
from tensorflow.contrib.distributions import Normal, Bernoulli
from tensorflow.contrib.tensorboard.plugins import projector

from sklearn.manifold import TSNE

import tensorflow_probability as tfp

tfd = tfp.distributions
tfb = tfp.bijectors

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

## Data

The data we use is regarding United Nations general debates (Kaggle.com, 2018). It has transcripts of general debates from 1970 to 2016. Each row in the data represents a speech given by a particular country’s representative in a particular year and session. The raw data has four columns: session, year, country and text. Session means the UN session and there is one session per year. Country means the country which the representative is from. Text means the transcript of the representative’s speech. <br>

Preprocessing: Since the data is in years, it is natural to set each time slice to be a year. All the speeches in the same year are attached together. In the resulting data folder, we have one txt file for each year, and each txt file includes all the speeches in that year. 

In [9]:
# Data is not included in the github repo because of its size. 
import pandas as pd
import numpy as np
data_df = pd.read_csv("un-general-debates.csv", encoding='utf-8')
num_speeches = []
vocab_total = set()
for single_year in range(1970, 2016):
    debate_single_year = data_df.loc[data_df.year == single_year]["text"]
    # print(len(debate_single_year))
    num_speeches.append(len(debate_single_year))
    debate_single_year = debate_single_year.str.cat()
    debate_single_year = debate_single_year.replace("\n", " ").replace("\t", " ")
    debate_single_year = debate_single_year.replace("   ", " ").replace("  ", " ")
    debate_single_year = debate_single_year.replace(",", "").replace(".", "")
    debate_single_year = debate_single_year.replace("?", "").lower()
    
    vocab_single_year = set(debate_single_year.split())
    vocab_total = vocab_total.union(vocab_single_year)
    
print("Total number of vocabulary is about " + str(len(vocab_total)))
print("\n")
print("Number of speeches in a year: \n" + str(pd.Series(num_speeches).describe()))
print("Total number of speeches in all years: " + str(sum(num_speeches)))

FileNotFoundError: File b'un-general-debates.csv' does not exist

As we can see from the above summary, there are about 70-195 speeches each year. The earlier years had less speeches and the later years had more speeches. There are 7507 speeches in the dataset in total. The number of unique words is roughly 100K. 

## Inference

We offer two inference methods for the clustering, Hamiltonian Monte Carlo (HMC) and Stochastic Gradient Descent Variational Inference (SGDVI) to get the Bayes estimate. For the embedding model, we employ stochastic gradient descent to get the MAP estimate.

### Hamiltonian Monte Carlo

<img src="hmc_ill.png"/>
(Illustration from Wang et al., 2019)

Hamiltonian Monte Carlo was proposed in the late 80s as a method to investigate the structure of particles (Duane et al., 1987). It then was borrowed into the statistics community as a very efficient method to approximate the posterior. The basic idea is that instead of moving randomly as we did in MH, we move around a vector field that we have some directions to follow to get to unexplored regions. We utilized the tensorflow probability to implement this as the following.

In [ ]:
class GMM_HMC(object):
    def __init__(
            self,
            observations,
            components=3,
            run_steps=1000,
            leap_steps=3,
            burn_in=100):
        # Create a mixture of two Gaussians:
        import tensorflow as tf
        import numpy as np
        import tensorflow_probability as tfp
        # from tensorflow_probability import edward2 as ed
        import functools

        tfd = tfp.distributions
        tfb = tfp.bijectors
        dtype = np.float32
        N, dims = observations.shape

        def joint_log_prob(observations, mix_probs, loc, chol_precision):
            rv_observations = tfd.MixtureSameFamily(
                mixture_distribution=tfd.Categorical(probs=mix_probs),
                components_distribution=MVNCholPrecisionTriL(
                    loc=loc,
                    chol_precision_tril=chol_precision))
            log_prob_parts = [
                rv_observations.log_prob(observations),  # Sum over samples.
                pi.log_prob(mix_probs)[..., tf.newaxis],
                mu.log_prob(loc),  # Sum over components.
                sigma.log_prob(chol_precision),  # Sum over components.
            ]
            sum_log_prob = tf.reduce_sum(
                tf.concat(log_prob_parts, axis=-1), axis=-1)
            # Note: for easy debugging, uncomment the following:
            # sum_log_prob = tf.Print(sum_log_prob, log_prob_parts)
            return sum_log_prob

        pi = tfd.Dirichlet(
            concentration=np.ones(components, dtype) * 1e3,
            name='pi_dist')
        mu = tfd.Independent(
            tfd.Normal(
                loc=np.stack([np.zeros(dims, dtype)] * components),
                scale=tf.ones([components, dims], dtype)),
            reinterpreted_batch_ndims=1,
            name='mu_dist')

        sigma = tfd.Wishart(
            df=dims,
            scale_tril=np.stack([np.eye(dims, dtype=dtype)] * components),
            input_output_cholesky=True,
            name='sigma_dist')

        # testing
        unnormalized_posterior_log_prob = functools.partial(
            joint_log_prob, observations)
        initial_state = [
            tf.fill([components],
                    value=np.array(1. / components, dtype),
                    name='mix_probs'),
            tf.constant(np.stack([np.zeros(dims, dtype)] * components),
                        name='mu'),
            tf.eye(dims, batch_shape=[components],
                   dtype=dtype, name='chol_precision'),
        ]
        unconstraining_bijectors = [
            tfb.SoftmaxCentered(),
            tfb.Identity(),
            tfb.Chain([
                tfb.TransformDiagonal(tfb.Softplus()),
                tfb.FillTriangular(),
            ])]
        with tf.variable_scope("alice_world", reuse=tf.AUTO_REUSE):
            mcstep_size = tf.get_variable(
                name='step_size',
                initializer=np.array(0.05, dtype),
                use_resource=True,
                trainable=False)
            [mix_probs, loc, chol_precision],\
                kernel_results = tfp.mcmc.sample_chain(
                    num_results=run_steps,
                    num_burnin_steps=burn_in,
                    current_state=initial_state,
                    kernel=tfp.mcmc.TransformedTransitionKernel(
                        inner_kernel=tfp.mcmc.HamiltonianMonteCarlo(
                            target_log_prob_fn=unnormalized_posterior_log_prob,
                            step_size=mcstep_size,
                            step_size_update_fn=tfp.mcmc.
                            make_simple_step_size_update_policy(),
                            num_leapfrog_steps=leap_steps),
                        bijector=unconstraining_bijectors))

            acceptance_rate = tf.reduce_mean(tf.to_float(
                kernel_results.inner_results.is_accepted))
            mean_mix_probs = tf.reduce_mean(mix_probs, axis=0)
            mean_loc = tf.reduce_mean(loc, axis=0)
            mean_chol_precision = tf.reduce_mean(chol_precision, axis=0)

            obs = tf.convert_to_tensor(observations)
            c = tf.get_variable("c",
                                [len(observations)],
                                dtype=dtype,
                                initializer=tf.zeros_initializer)
            c_log = tf.get_variable("c_log",
                                    [len(observations)],
                                    dtype=dtype,
                                    initializer=tf.zeros_initializer)
            # tf.get_variable_scope().reuse_variables()

            def condition(i, px): return i < len(observations)

            def c_assign(i, c):
                theta = tfd.MultivariateNormalTriL(
                    loc=mean_loc, scale_tril=mean_chol_precision)
                c_log = tf.get_variable("c_log")
                c_log = tf.scatter_update(
                    c_log,
                    indices=i,
                    updates=tf.reduce_max(
                        tf.log(mean_mix_probs) +
                        theta.log_prob(
                            obs[i])))
                c = tf.get_variable("c")
                c = tf.scatter_update(
                    c,
                    indices=i,
                    updates=tf.to_float(
                        tf.argmax(
                            tf.log(mean_mix_probs) +
                            theta.log_prob(
                                obs[i]))))
                return i + 1, c

            i, c = tf.while_loop(condition, c_assign, [0, c])
        with tf.Session() as sess:
            init_op = tf.global_variables_initializer()
            sess.run(init_op)
            # sess.run(init_op)
            [
                self.acceptance_rate_,
                self.pi_out,
                self.mu_out,
                self.sigma_out,
                self.assignment,
                self.assignment_likelihood
            ] = sess.run([
                acceptance_rate,
                mean_mix_probs,
                mean_loc,
                mean_chol_precision,
                c,
                c_log
            ])

### Stochastic Gradient Descent Variational Inference

Due to our setting, we can make the conjugate exponential family (CEF) argument such that under the mean-field assumption, the variational distributions will be in the same form as the distributions that they are trying to approximate.

$$
\begin{align*}
p(\pi,\mu,\Lambda, c|\rho) \approx q(\pi)[\prod_{j=1}^{K} q(\mu_j)q(\Lambda_j)][\prod_{i=1}^{n} q(c_i)]
\end{align*}$$

$$
\begin{align*}
q(\pi) &= Dir(\alpha_1', \alpha_2',...,\alpha_k') \\
q(\mu_j) &= N(m_j',\Sigma_j') \\
q(\Lambda_j) &= Wishart(a_j',B_j')  \\
q(c_i) &= Discrete(\phi_i) 
\end{align*}
$$

The ELBO is then
$$
L = E_q[\ln p(\rho,\pi,\mu,\Lambda, c) - \ln q ] + const.
$$

Analytical optimal update schemes can be derived, but since we are dealing with high-dimensional big data, we choose to use SGD methods instead. 

The two methods are both implemented as classes.

## Criticizm

Since we built an embedding model and a topic model at the same time, we will evaluate it in two metrics, the log-likelihood and the topic coherence. 

For the log-likelihood, we compare the likelihood for the positive samples among our model, the dynamic Bernoulli embedding model, and the Bernoulli embedding model.

Regarding the topic coherence, it is difficult to define it explicitly so there are multiple indicators come from different perspectives.

### Experiment Setting

We train on top **10000 words, with 5000 training steps, 10 mixture components and embedding dimension of 50**.

### Training Process and log-likelihood

#### Our Model (with HMC)
<img src="DCEMB_LOSS.png" style="display:inline;margin:1px;width:49%;" />
<img src="DCEMB_MIX.png"  style="display:inline;margin:1px;width:49%;"/>
#### Dynamic Embedding Model and Bournoulli Embedding Model
<img src="DEMB_LOSS.png"  style="display:inline;margin:1px;width:49%;"/>
<img src="EMB_LOSS.png"   style="display:inline;margin:1px;width:49%;"/>

### Topic Coherence

#### $C_{uci}$

It measures the co-occurrence probability of every word pairs in a topic (Newman et al., 2010).
Suppose we have a topic of three words {one, two, three}. The co-occurrence probability of any two words would be calculated based on sliding windows, for example, if our text is "One is Two", the virtual documents with a size 2 sliding window would be {"One is", "is Two"}.

In this case, P(one) = $\frac{1}{2}$  (appeared once in two virtual documents), P(one, two) = 0 (no co-occurrence of one and two)
$$PMI(one, two) = \frac{log(P(one,two)+\epsilon)}{P(one)\cdot P(two)} $$($\epsilon$ is for smoothing, otherwise we get log(0))

$$C_{uci} = \frac{1}{3}\cdot[PMI(one, two) + PMI(one, three) + PMI(two, three)]$$

#### $U_{mass}$

It measures the conditional probability of weaker words given stronger words in a topic (Mimno et al., 2011). The idea is that the occurrence of every top word should be supported by every preceding top word.
Followed by our previous {one, two, three} topic.

$$U_{mass}  = \frac{1}{3}\cdot( K(two|one) + K(three|one) + K(three|two))$$ where $$K(two|one) = \frac{log (P(two, one) + 1)}{P(one)}$$

#### $C_{npmi}$

It is an improved version of $C_{uci}$ by adding normalization (Aletras & Stevenson, 2013).
It is just like $C_{uci}$ but changed PMI to NPMI (normalized PMI), which is listed below:
$$NPMI(one, two)  = (\frac{\frac{log( P(one,two)+1)}{P(one)\cdot P(two)}}{-log(P(one,two)+1)})^\gamma $$
(An increase of $\gamma$ gives higher NPMI values more weight, $\gamma$ = 1 is a common choice)

#### $C_v$

It measures the indirect similarity among words in a topic (Röder et al., 2015).
Some words belong to the same topic but they rarely occur together. But the words around them should be similar. For example, suppose we have "McDonald makes chicken nuggets" and "KFC serves chicken nuggets", we will probably want to put McDonald and KFC together. That's the intuition behind indirect similarity.

However, the math is a bit complicated here.

The co-occurrence counts are used to calculated the NPMI of every top word to every other top word, thus, resulting in a set of vectors, one for every top word. The calculation of the similarity is done between every top word vector and the sum of all top word vectors. As similarity measure, the cosinus is used.

| Coherence Measure | DCEMB   |   LDA   |
|------|------|------|
|   $C_{uci}$  | -7.8974 |**-0.1428**|
|   $U_{mass}$  | -6.7553 |**-0.0001**|
|   $C_{npmi}$  | -0.2838 |**-0.0388**|
|   $C_v$  | **0.4300** |0.2892|

### Qualitative Exploration

We explore the resulted topics as well as the contextual change of words over time.

#### Topic Visualization aftering Mapping to $\mathbb{R}^2$ with TSNE

<img src="rho_1.png" style="width:110%;">
<img src="rho.png" style="display:inline;margin:1px;width:110%;">

#### Top 10 words for each topic

| Topic 1 | Topic 2  |   Topic 3  |Topic 4 | Topic 5   |   Topic 6   |Topic 7 | Topic 8   |   Topic 9   |Topic 10|
|------|------|------|------|------|------|------|------|------|------|
|   fruition | lome | unfulfilled |  andean | subjugation |leone|   tensions  | korean |strongly|   the  |
|  migratory  | forceful |uncontrolled| candidature |culminated |denied| achievements  | concept |critical|   of |
|  minute  | stature |unjustified| gloomy | obtaining |training|  attempts  | protect|obligations|   and  |
|   constituting | commits |spiral|  optional| electricity |white|  addition  | moral |reiterate|  to  |
|   daughters  | sadat |blacks| unanimity  |amaral |quickly|   somalia  | court |causes|  in  |
|  prudence | dividends |dictate| thant  | differing |bolivia|  borders | millions |participate|   a  |
|  champions | thorn |pernicious|  expansionism | agendas |sister|  constitute | around |representatives|   that |
|  com  | interpretations |deplores|  persisted  | wasted |inhabitants|  signed | grave |above|   is  |
|  edge | rescheduling |graduation|  nationally  |flourish |destroy|  weapon  | light |presence|  for  |
|  impatience | nassir |aviv|  steer  | understandable |eradicate|  stage | discrimination |outside|   we |

## Reference

Wang, Z., M. Broccardo, and J. Song* (2019). Hamiltonian Monte Carlo methods for subset simulation in reliability analysis. Structural Safety. Vol. 76, 51-67

Rudolph, M., & Blei, D. (2018, April). Dynamic Embeddings for Language Evolution. In Proceedings of the 2018 World Wide Web Conference on World Wide Web (pp. 1003-1011). International World Wide Web Conferences Steering Committee.

Rudolph, M., Ruiz, F., Mandt, S., & Blei, D. (2016). Exponential family embeddings. In Advances in Neural Information Processing Systems (pp. 478-486).

Kaggle.com. (2018). UN General Debates. [online] Available at: https://www.kaggle.com/unitednations/un-general-debates [Accessed 27 Nov. 2018].

Duane, S., Kennedy, A. D., Pendleton, B. J., & Roweth, D. (1987). Hybrid monte carlo. Physics letters B, 195(2), 216-222.

Newman, D., Lau, J. H., Grieser, K., & Baldwin, T. (2010, June). Automatic evaluation of topic coherence. In Human Language Technologies: The 2010 Annual Conference of the North American Chapter of the Association for Computational Linguistics (pp. 100-108). Association for Computational Linguistics.

Mimno, D., Wallach, H. M., Talley, E., Leenders, M., & McCallum, A. (2011, July). Optimizing semantic coherence in topic models. In Proceedings of the conference on empirical methods in natural language processing (pp. 262-272). Association for Computational Linguistics.

Aletras, N., & Stevenson, M. (2013). Evaluating topic coherence using distributional semantics. In Proceedings of the 10th International Conference on Computational Semantics (IWCS 2013)–Long Papers (pp. 13-22).

Röder, M., Both, A., & Hinneburg, A. (2015, February). Exploring the space of topic coherence measures. In Proceedings of the eighth ACM international conference on Web search and data mining (pp. 399-408). ACM.